In [1]:
import boto3
import pandas as pd
import numpy as np

In [2]:
def download_file_from_s3(bucket, object_name, local_file_name):
    """
    Download a file from S3 to the local file system.

    :param bucket: Name of the S3 bucket
    :param object_name: S3 object name
    :param local_file_name: Local file name to save the downloaded file
    """
    # # When using IAM roles, boto3 retrieves credentials from the instance metadata
    # s3_client = boto3.client('s3')

    #When setting up credentials locally, use the following code
    session = boto3.Session()
    s3_client = session.client('s3')

    s3_client.download_file(bucket, object_name, local_file_name)

bucket_name = 'myukdata'
s3_file_name = 'Original/Member/Member_PartyAffiliations.csv'
local_file = 'Member_PartyAffiliations.csv'

download_file_from_s3(bucket_name, s3_file_name, local_file)

In [3]:
df = pd.read_csv('Member_PartyAffiliations.csv')
df.info()
pd.set_option('display.max_rows', None)
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5457 entries, 0 to 5456
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   memberID            5457 non-null   int64  
 1   house               5407 non-null   float64
 2   name                5457 non-null   object 
 3   id                  5457 non-null   int64  
 4   startDate           5457 non-null   object 
 5   endDate             3988 non-null   object 
 6   additionalInfo      0 non-null      float64
 7   additionalInfoLink  0 non-null      float64
dtypes: float64(3), int64(2), object(3)
memory usage: 341.2+ KB


,memberID,house,name,id,startDate,endDate,additionalInfo,additionalInfoLink
0,1,1.0,Labour,15,1997-05-01T00:00:00,2010-05-06T00:00:00,NaN,NaN
1,2,1.0,Independent,8,2017-11-02T00:00:00,2019-11-06T00:00:00,NaN,NaN
2,2,1.0,Labour,15,1997-05-01T00:00:00,2017-11-02T00:00:00,NaN,NaN
3,3,1.0,Independent,8,2010-03-22T00:00:00,2010-05-06T00:00:00,NaN,NaN
4,3,1.0,Labour,15,1997-05-01T00:00:00,2010-03-22T00:00:00,NaN,NaN


In [4]:
df.drop(columns=['additionalInfo',
                 'additionalInfoLink',
                 'house'], inplace=True)

df.rename(columns={'memberID': 'memberId',
                     'name': 'partyName',
                     'id':'partyId'}, inplace=True)
df['startDate'] = pd.to_datetime(df['startDate'])
df['endDate'] = pd.to_datetime(df['endDate'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5457 entries, 0 to 5456
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   memberId   5457 non-null   int64         
 1   partyName  5457 non-null   object        
 2   partyId    5457 non-null   int64         
 3   startDate  5457 non-null   datetime64[ns]
 4   endDate    3988 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(1)
memory usage: 213.3+ KB


In [6]:
#Save the file to S3
import boto3
import io

def upload_df_to_s3(df, bucket, object_name):
    """
    Upload a DataFrame to an S3 bucket as CSV.

    :param df: DataFrame to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name
    :return: True if the DataFrame was uploaded, else False
    """
    # Create a buffer
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer, index=False)

    # Move to the start of the buffer
    csv_buffer.seek(0)

    # Upload the buffer content to S3
    s3_client = boto3.client('s3')
    try:
        s3_client.put_object(Bucket=bucket, Key=object_name, Body=csv_buffer.getvalue())
    except ClientError as e:
        logging.error(e)
        return False
    return True

bucket_name = 'myukdata'
object_name = 'Preprocessing/Member/MemberPartyAffiliations.csv'
upload_df_to_s3(df, bucket_name, object_name)


True